In [118]:
# Generate topics for class in JDT

# configuration - begin
NUM_TOPIC = 5
inputFile = "C:\\Users\\Duc Anh Nguyen\\Desktop\\jdt_document.csv"
outputFile = 'C:\\Users\\Duc Anh Nguyen\\Desktop\\jdt_topic.csv'
# configuration - end

import csv

filename = []
loc = []
noc = []
bf = []
bug = []
documents = []

with open(inputFile) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            filename.append(row[0])
            loc.append(row[1])
            noc.append(row[2])
            bf.append(row[3])
            bug.append(row[4])
            documents.append(row[5])
            line_count += 1
    print(f'Processed {line_count} lines.')
    
print(documents[0])

Column names are file, loc, noc, bf, bug, document
Processed 998 lines.
internal core search indexing index binary folder 


In [119]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Duc Anh
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Duc Anh
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [120]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in documents] 
doc_clean[0]

['internal', 'core', 'search', 'indexing', 'index', 'binary', 'folder']

In [121]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=NUM_TOPIC, id2word = dictionary, passes=50)

ldamodel.print_topics(-1)

[(0,
  '0.184*"internal" + 0.133*"compiler" + 0.055*"core" + 0.035*"util" + 0.031*"lookup" + 0.026*"info" + 0.024*"ast" + 0.023*"binding" + 0.021*"builder" + 0.018*"source"'),
 (1,
  '0.140*"internal" + 0.079*"codeassist" + 0.060*"rewrite" + 0.058*"formatter" + 0.058*"comment" + 0.050*"selection" + 0.046*"select" + 0.034*"import" + 0.028*"node" + 0.025*"impl"'),
 (2,
  '0.214*"core" + 0.138*"dom" + 0.065*"internal" + 0.034*"operation" + 0.033*"expression" + 0.023*"declaration" + 0.022*"statement" + 0.022*"package" + 0.021*"compilation" + 0.019*"member"'),
 (3,
  '0.151*"internal" + 0.122*"core" + 0.067*"search" + 0.053*"type" + 0.046*"compiler" + 0.042*"ast" + 0.035*"reference" + 0.031*"matching" + 0.020*"java" + 0.016*"declaration"'),
 (4,
  '0.149*"internal" + 0.073*"completion" + 0.069*"codeassist" + 0.061*"core" + 0.059*"complete" + 0.039*"code" + 0.036*"eval" + 0.027*"index" + 0.024*"snippet" + 0.023*"indexing"')]

In [122]:
# This is example
# Get the best topic of a document
document_id = 1
x = ldamodel[doc_term_matrix[document_id]]
highestProbTopic = 0
highestTopic = 0
print(x)
for i in x:
    if highestProbTopic < i[1] :
        highestProbTopic = i[1]
        highestTopic = i[0]

print(highestProbTopic)
print(highestTopic)

[(0, 0.5397598), (1, 0.028693667), (2, 0.028628027), (3, 0.20931694), (4, 0.1936016)]
0.5397598
0


In [135]:
# Export to csv file
with open(outputFile, mode='w', newline='') as FILE:
    FILE_writer = csv.writer(FILE, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    # Create headers
    headers = ["file", "loc", "noc", "bf", "bug", "document", "topic"]
    for i in range(0, NUM_TOPIC):
            headers.append("topic" + str(i))
            
    FILE_writer.writerow(headers)
    #
    for i in range(0, len(filename)) :
        # Find the best topic
        predict = ldamodel[doc_term_matrix[i]]
        highestProbTopic = 0
        bestTopic = 0
        for topic in predict:
            if highestProbTopic < topic[1] :
                highestProbTopic = topic[1]
                bestTopic = topic[0]
        
        # write
        output = [filename[i], loc[i], noc[i], bf[i], bug[i], documents[i], bestTopic]
        for i in range(0, bestTopic):
            output.append(0)
        
        output.append(1)
        
        for i in range(bestTopic+1, NUM_TOPIC):
            output.append(0)
            
        FILE_writer.writerow(output)
    
    print("done")

done
